# IEEE-CIS Fraud Detection step1: Data Wrangling

Link to Data: https://www.kaggle.com/c/ieee-fraud-detection/overview

## Data information
### Transaction Table *
- TransactionDT: timedelta from a given reference datetime (not an actual timestamp)
- TransactionAMT: transaction payment amount in USD
- ProductCD: product code, the product for each transaction
- card1 - card6: payment card information, such as card type, card category, issue bank, country, etc.
- addr: address
- dist: distance
- P_ and (R__) emaildomain: purchaser and recipient email domain
- C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
- D1-D15: timedelta, such as days between previous transaction, etc.
- M1-M9: match, such as names on card and address, etc.
- Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.

#### Categorical Features:
- ProductCD
- card1 - card6
- addr1, addr2
- Pemaildomain Remaildomain
- M1 - M9
----------------------------------------------------------------

### Identity Table*
- Variables in this table are identity information – network connection information (IP, ISP, Proxy, etc) and digital signature (UA/browser/os/version, etc) associated with transactions.
- They're collected by Vesta’s fraud protection system and digital security partners. (The field names are masked and pairwise dictionary will not be provided for privacy protection and contract agreement)

#### Categorical Features:
- DeviceType
- DeviceInfo
- id12 - id38

## Import Library

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')

In [2]:
!ls

Chi Hyun Nam - Capstone 2 Project Ideas.gdoc
Step_one_datawrangling.ipynb
data
figures
models
sample_submission.csv
test_identity.csv
test_transaction.csv
train_identity.csv
train_transaction.csv


In [3]:
path = os.getcwd()
print("current working directory: ", path)

current working directory:  C:\Users\chi\Desktop\spring_board\capstone_2


## Create subfolder in project directory

In [4]:
os.makedirs('data',exist_ok=True)
os.makedirs('figures',exist_ok=True)
os.makedirs('models',exist_ok=True)

In [5]:
!ls

Chi Hyun Nam - Capstone 2 Project Ideas.gdoc
Step_one_datawrangling.ipynb
data
figures
models
sample_submission.csv
test_identity.csv
test_transaction.csv
train_identity.csv
train_transaction.csv


### Load Data from the csv file

In [2]:
transaction_df = pd.read_csv('train_transaction.csv')
identity_df = pd.read_csv('train_identity.csv')

# Clean transaction data
The transaction data will be cleaned first

In [7]:
transaction_df.head()
# There is lots of nan values in the dataset
# TransactionID is primarykey for the dataset

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
transaction_df.tail()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
590535,3577535,0,15811047,49.00,W,6550,NaN,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,3577536,0,15811049,39.50,W,10444,225.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,3577537,0,15811079,30.95,W,12037,595.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590538,3577538,0,15811088,117.00,W,7826,481.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590539,3577539,0,15811131,279.95,W,15066,170.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
transaction_df.shape

(590540, 394)

In [10]:
transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Columns: 394 entries, TransactionID to V339
dtypes: float64(376), int64(4), object(14)
memory usage: 1.7+ GB


In [11]:
transaction_df.describe()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
count,5.905400e+05,590540.000000,5.905400e+05,590540.000000,590540.000000,581607.000000,588975.000000,586281.000000,524834.000000,524834.000000,...,82351.000000,82351.000000,82351.000000,82351.000000,82351.000000,82351.00000,82351.000000,82351.000000,82351.000000,82351.000000
mean,3.282270e+06,0.034990,7.372311e+06,135.027176,9898.734658,362.555488,153.194925,199.278897,290.733794,86.800630,...,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
std,1.704744e+05,0.183755,4.617224e+06,239.162522,4901.170153,157.793246,11.336444,41.244453,101.741072,2.690623,...,4.727971,6217.223583,11169.275702,7955.735482,243.861391,387.62948,274.576920,668.486833,1095.034387,814.946722
min,2.987000e+06,0.000000,8.640000e+04,0.251000,1000.000000,100.000000,100.000000,100.000000,100.000000,10.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,3.134635e+06,0.000000,3.027058e+06,43.321000,6019.000000,214.000000,150.000000,166.000000,204.000000,87.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,3.282270e+06,0.000000,7.306528e+06,68.769000,9678.000000,361.000000,150.000000,226.000000,299.000000,87.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
75%,3.429904e+06,0.000000,1.124662e+07,125.000000,14184.000000,512.000000,150.000000,226.000000,330.000000,87.000000,...,0.000000,0.000000,25.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
max,3.577539e+06,1.000000,1.581113e+07,31937.391000,18396.000000,600.000000,231.000000,237.000000,540.000000,102.000000,...,55.000000,160000.000000,160000.000000,160000.000000,55125.000000,55125.00000,55125.000000,104060.000000,104060.000000,104060.000000


In [12]:
transaction_df.isnull().sum().sort_values(ascending = False)[transaction_df.isnull().sum() > 0]
# There are 374 columns with null values

dist2    552913
D7       551623
D13      528588
D14      528353
D12      525823
          ...  
V321         12
V299         12
V307         12
V306         12
V309         12
Length: 374, dtype: int64

In [13]:
len(transaction_df.isnull().sum().sort_values(ascending = False)[transaction_df.isnull().sum() > 500000])

55

In [14]:
transaction_df.isnull().sum().sort_values(ascending = False)[transaction_df.isnull().sum() > 500000] * 100 / transaction_df.shape[0]
# dist2 implies distance between transaction
# D# implies timedelta between transaction
# V## implies Vesta engineered rich features, including ranking, counting, and other entity relations

# These variable looks all important
# It's better not to dropp all the data the Nan values will be replaced with 0

dist2    93.628374
D7       93.409930
D13      89.509263
D14      89.469469
D12      89.041047
D6       87.606767
D8       87.312290
D9       87.312290
V142     86.123717
V157     86.123717
V153     86.123717
V154     86.123717
V155     86.123717
V156     86.123717
V163     86.123717
V158     86.123717
V161     86.123717
V162     86.123717
V148     86.123717
V149     86.123717
V147     86.123717
V141     86.123717
V138     86.123717
V146     86.123717
V139     86.123717
V140     86.123717
V166     86.122701
V164     86.122701
V160     86.122701
V159     86.122701
V165     86.122701
V150     86.122701
V143     86.122701
V144     86.122701
V145     86.122701
V152     86.122701
V151     86.122701
V338     86.054967
V339     86.054967
V333     86.054967
V322     86.054967
V336     86.054967
V335     86.054967
V334     86.054967
V337     86.054967
V332     86.054967
V330     86.054967
V331     86.054967
V323     86.054967
V324     86.054967
V325     86.054967
V326     86.054967
V327     86.

## Nan Value
- The dataset contains too many nan values the columns with high nan cannot be dropped there is just too many and each have important feature. The Nan values will be replaced with 0 in the following columns.<br>
#### dist, C1-C14, D1-D15, M1-M9, Vxxx

In [15]:
# import regex experession
import re

In [16]:
column_list = [*transaction_df.columns]

def find_from(mylist, exp):
    r = re.compile(exp)
    newlist = list(filter(r.match, mylist))
    return newlist

In [17]:
fill_col = []
fill_col.extend(find_from(column_list, "dist+"))
fill_col.extend(find_from(column_list, "C+"))
fill_col.extend(find_from(column_list, "D+"))
fill_col.extend(find_from(column_list, "V+"))
fill_col.extend(find_from(column_list, "M+"))
print(fill_col)

['dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V107', 'V108', 'V109', 'V110', 'V111', 'V112', 'V113', 

In [18]:
transaction_df[fill_col] = transaction_df[fill_col].fillna(0.0)
# fill 0.0 in extracted columns

In [19]:
transaction_df.isnull().sum().sort_values(ascending = False)[transaction_df.isnull().sum() > 0] 

R_emaildomain    453249
P_emaildomain     94456
addr2             65706
addr1             65706
card2              8933
card5              4259
card4              1577
card6              1571
card3              1565
dtype: int64

Now there are only 9 columns with null values

___

In [20]:
# null value in percentage
100 * transaction_df.isnull().sum().sort_values(ascending = False)[transaction_df.isnull().sum() > 0] / transaction_df.shape[0]

R_emaildomain    76.751617
P_emaildomain    15.994852
addr2            11.126427
addr1            11.126427
card2             1.512683
card5             0.721204
card4             0.267044
card6             0.266028
card3             0.265012
dtype: float64

In [21]:
column_names = transaction_df.isnull().sum().sort_values(ascending = False)[transaction_df.isnull().sum() > 0].index
transaction_df[column_names].head()
# These are all categorical data

,R_emaildomain,P_emaildomain,addr2,addr1,card2,card5,card4,card6,card3
0,NaN,NaN,87.0,315.0,NaN,142.0,discover,credit,150.0
1,NaN,gmail.com,87.0,325.0,404.0,102.0,mastercard,credit,150.0
2,NaN,outlook.com,87.0,330.0,490.0,166.0,visa,debit,150.0
3,NaN,yahoo.com,87.0,476.0,567.0,117.0,mastercard,debit,150.0
4,NaN,gmail.com,87.0,420.0,514.0,102.0,mastercard,credit,150.0


In [22]:
transaction_df[column_names].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   R_emaildomain  137291 non-null  object 
 1   P_emaildomain  496084 non-null  object 
 2   addr2          524834 non-null  float64
 3   addr1          524834 non-null  float64
 4   card2          581607 non-null  float64
 5   card5          586281 non-null  float64
 6   card4          588963 non-null  object 
 7   card6          588969 non-null  object 
 8   card3          588975 non-null  float64
dtypes: float64(5), object(4)
memory usage: 40.5+ MB


In [23]:
# fill Nan value as "NI" as No Information, the null values on catagorical variables cannot be just dropped because they have null value
# it can be the reason of not applicable reason or does not have value to put them in
transaction_df[column_names] = transaction_df[column_names].fillna('NI')

In [24]:
# There is nomore nan value
100 * transaction_df.isnull().sum().sort_values(ascending = False)[transaction_df.isnull().sum() > 0] / transaction_df.shape[0]

Series([], dtype: float64)

## Change datatype to object for categorical variables
- ProductCD
- card1 - card6
- addr1, addr2
- Pemaildomain Remaildomain
- M1 - M9

In [25]:
cat_col = [*column_names]
m_col=[*transaction_df.select_dtypes(include=['object']).columns]
cat_col.append('ProductCD')
cat_col.append('card1')
cat_col.extend(m_col)
cat_col = list(set(cat_col))
cat_col.sort()
print(cat_col)

['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'P_emaildomain', 'ProductCD', 'R_emaildomain', 'addr1', 'addr2', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6']


In [26]:
transaction_df[cat_col] = transaction_df[cat_col].astype(str)

In [27]:
transaction_df[cat_col].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   M1             590540 non-null  object
 1   M2             590540 non-null  object
 2   M3             590540 non-null  object
 3   M4             590540 non-null  object
 4   M5             590540 non-null  object
 5   M6             590540 non-null  object
 6   M7             590540 non-null  object
 7   M8             590540 non-null  object
 8   M9             590540 non-null  object
 9   P_emaildomain  590540 non-null  object
 10  ProductCD      590540 non-null  object
 11  R_emaildomain  590540 non-null  object
 12  addr1          590540 non-null  object
 13  addr2          590540 non-null  object
 14  card1          590540 non-null  object
 15  card2          590540 non-null  object
 16  card3          590540 non-null  object
 17  card4          590540 non-null  object
 18  card

## Find Duplicated

In [28]:
# There is no duplicated value
transaction_df.duplicated().sum()

0

# Save transaction data

In [29]:
transaction_df.to_csv('C:\\Users\\chi\\Desktop\\spring_board\\capstone_2\\data\\Clean_transaction.csv', index=False)

In [30]:
clean = pd.read_csv('C:\\Users\\chi\\Desktop\\spring_board\\capstone_2\\data\\Clean_transaction.csv')

C:\Users\chi\Anaconda3\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
clean[cat_col]._get_numeric_data().info()
# Aware that changed data type on 'card1' column is not saved as object data type
# CSV format does not store information about data type
# Use code below to load data
# saved_df = pd.read_csv('C:\\Users\\chi\\Desktop\\spring_board\\capstone_2\\data\\Clean_transaction.csv', dtype={'card1': object})

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   card1   590540 non-null  int64
dtypes: int64(1)
memory usage: 4.5 MB


In [32]:
clean[cat_col]._get_numeric_data().head()

,card1
0,13926
1,2755
2,4663
3,18132
4,4497


In [33]:
[*clean[cat_col]._get_numeric_data().columns]

['card1']

In [34]:
clean.isnull().sum()[clean.isnull().sum() > 0]

Series([], dtype: int64)

___

# Clean Identity data

The Identity data will be merged with transaction data after cleaning process

In [3]:
identity_df.head()
# There are 41 columns in the dataset
# it is clear that dataset contains lots of null values

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [4]:
identity_df.tail()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
144228,3577521,-15.0,145955.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 66.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,F3111 Build/33.3.A.1.97
144229,3577526,-5.0,172059.0,NaN,NaN,1.0,-5.0,NaN,NaN,NaN,...,chrome 55.0 for android,32.0,855x480,match_status:2,T,F,T,F,mobile,A574BL Build/NMF26F
144230,3577529,-20.0,632381.0,NaN,NaN,-1.0,-36.0,NaN,NaN,NaN,...,chrome 65.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,Moto E (4) Plus Build/NMA26.42-152
144231,3577531,-5.0,55528.0,0.0,0.0,0.0,-7.0,NaN,NaN,0.0,...,chrome 66.0,24.0,2560x1600,match_status:2,T,F,T,F,desktop,MacOS
144232,3577534,-45.0,339406.0,NaN,NaN,-10.0,-100.0,NaN,NaN,NaN,...,chrome 66.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,RNE-L03 Build/HUAWEIRNE-L03


In [5]:
identity_df.shape
# There are 144233 rows and 41 columns in the dataset

(144233, 41)

In [6]:
identity_df.info()
# there are 24 numerical and 17 object data types
# There are 29 categorical variables in the dataset

# Categorical: 
# DeviceType
# DeviceInfo
# id12 - id38

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144233 entries, 0 to 144232
Data columns (total 41 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TransactionID  144233 non-null  int64  
 1   id_01          144233 non-null  float64
 2   id_02          140872 non-null  float64
 3   id_03          66324 non-null   float64
 4   id_04          66324 non-null   float64
 5   id_05          136865 non-null  float64
 6   id_06          136865 non-null  float64
 7   id_07          5155 non-null    float64
 8   id_08          5155 non-null    float64
 9   id_09          74926 non-null   float64
 10  id_10          74926 non-null   float64
 11  id_11          140978 non-null  float64
 12  id_12          144233 non-null  object 
 13  id_13          127320 non-null  float64
 14  id_14          80044 non-null   float64
 15  id_15          140985 non-null  object 
 16  id_16          129340 non-null  object 
 17  id_17          139369 non-nul

In [7]:
# id_01 - id_11 is the only numeric data
numeric = []

for i in range(1, 12):
    if i < 10:
        numeric.append('id_0{}'.format(str(i)))
    else:
        numeric.append('id_{}'.format(str(i)))

print(numeric)

['id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 'id_09', 'id_10', 'id_11']


In [8]:
identity_df[numeric].describe()

,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11
count,144233.000000,140872.000000,66324.000000,66324.000000,136865.000000,136865.000000,5155.000000,5155.000000,74926.000000,74926.000000,140978.000000
mean,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.698710,13.285354,-38.600388,0.091023,-0.301124,99.745325
std,14.347949,159651.816856,0.598231,0.701015,5.249856,16.491104,11.384207,26.084899,0.983842,2.789446,1.127602
min,-100.000000,1.000000,-13.000000,-28.000000,-72.000000,-100.000000,-46.000000,-100.000000,-36.000000,-100.000000,90.000000
25%,-10.000000,67992.000000,0.000000,0.000000,0.000000,-6.000000,5.000000,-48.000000,0.000000,0.000000,100.000000
50%,-5.000000,125800.500000,0.000000,0.000000,0.000000,0.000000,14.000000,-34.000000,0.000000,0.000000,100.000000
75%,-5.000000,228749.000000,0.000000,0.000000,1.000000,0.000000,22.000000,-23.000000,0.000000,0.000000,100.000000
max,0.000000,999595.000000,10.000000,0.000000,52.000000,0.000000,61.000000,0.000000,25.000000,0.000000,100.000000


## Nan Value

In [9]:
100 * identity_df.isnull().sum().sort_values(ascending =False)[identity_df.isnull().sum() > 0] / identity_df.shape[0]
# There are 38 columns with null values
# The values are in percentage of null values

id_24         96.708798
id_25         96.441868
id_07         96.425922
id_08         96.425922
id_21         96.423149
id_26         96.420375
id_22         96.416215
id_27         96.416215
id_23         96.416215
id_18         68.722137
id_04         54.016071
id_03         54.016071
id_33         49.187079
id_10         48.052110
id_09         48.052110
id_30         46.222432
id_32         46.207872
id_34         46.056034
id_14         44.503685
DeviceInfo    17.726179
id_13         11.726165
id_16         10.325654
id_06          5.108401
id_05          5.108401
id_20          3.447200
id_19          3.407681
id_17          3.372321
id_31          2.739318
DeviceType     2.373243
id_02          2.330257
id_11          2.256765
id_28          2.256765
id_29          2.256765
id_15          2.251912
id_35          2.251912
id_36          2.251912
id_37          2.251912
id_38          2.251912
dtype: float64

In [10]:
# Let's see numerical variables first
# id01 to id11 are numerical features for identity, 
# which is collected by Vesta and security partners such as device rating, ip_domain rating, proxy rating, etc.
100 * identity_df[numeric].isnull().sum() / identity_df.shape[0]

id_01     0.000000
id_02     2.330257
id_03    54.016071
id_04    54.016071
id_05     5.108401
id_06     5.108401
id_07    96.425922
id_08    96.425922
id_09    48.052110
id_10    48.052110
id_11     2.256765
dtype: float64

In [11]:
print(identity_df['id_07'].unique(),'\n')
print(sorted(identity_df['id_08'].unique()))

[ nan  22.   6.  -1.   4.   2.  17.  12.  27.   5.  18.  16.  23.  -4.
  25.  20.  39.  21.   0.  -9.  10.   1.  19.  24.  29.  13.  30.  11.
   8.   7.  14.  31.  32.   9.  -5.   3.  26.  28.  33.  15.  -6.  -2.
  38.  34.  37. -16. -14.  -8.  41. -10.  -3.  -7.  36.  35. -13. -12.
 -11.  48.  40. -19.  43.  42. -18.  61.  44. -23.  52.  45.  51.  49.
 -46. -24. -27. -22. -31. -21. -17. -32. -33. -37. -30.  46. -26. -15.
 -20.] 

[nan, -100.0, -99.0, -98.0, -97.0, -95.0, -93.0, -92.0, -90.0, -88.0, -85.0, -84.0, -82.0, -81.0, -80.0, -79.0, -78.0, -77.0, -76.0, -75.0, -74.0, -73.0, -72.0, -71.0, -70.0, -69.0, -68.0, -67.0, -66.0, -65.0, -64.0, -63.0, -62.0, -61.0, -60.0, -59.0, -58.0, -57.0, -56.0, -55.0, -54.0, -53.0, -52.0, -51.0, -50.0, -49.0, -48.0, -47.0, -46.0, -45.0, -44.0, -43.0, -42.0, -41.0, -40.0, -39.0, -38.0, -37.0, -36.0, -35.0, -34.0, -33.0, -32.0, -31.0, -30.0, -29.0, -28.0, -27.0, -26.0, -25.0, -24.0, -23.0, -22.0, -21.0, -20.0, -19.0, -18.0, -17.0, -16.0, -15.0, -14.0

In [12]:
# Let's drop id_08, id_07 columns first there is too many null values
identity_df.drop(columns=['id_07', 'id_08'], inplace = True)

#### column 'id_07', 'id_08' is dropped

In [13]:
# Let's work on 'id_03', 'id_04', 'id_09', 'id_10' columns
cols = ['id_03', 'id_04', 'id_09', 'id_10']

identity_df[cols].describe()

,id_03,id_04,id_09,id_10
count,66324.000000,66324.000000,74926.000000,74926.000000
mean,0.060189,-0.058938,0.091023,-0.301124
std,0.598231,0.701015,0.983842,2.789446
min,-13.000000,-28.000000,-36.000000,-100.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000
max,10.000000,0.000000,25.000000,0.000000


In [14]:
# their first, second , third percentile is all 0.0
# assume null as 0.0
identity_df[cols] = identity_df[cols].fillna(0.0)

####  'id_03', 'id_04', 'id_09', 'id_10' columns nulls are filled with 0.0

In [15]:
lowna_cols = ['id_02','id_05', 'id_06', 'id_11']
identity_df[lowna_cols].describe()

,id_02,id_05,id_06,id_11
count,140872.000000,136865.000000,136865.000000,140978.000000
mean,174716.584708,1.615585,-6.698710,99.745325
std,159651.816856,5.249856,16.491104,1.127602
min,1.000000,-72.000000,-100.000000,90.000000
25%,67992.000000,0.000000,-6.000000,100.000000
50%,125800.500000,0.000000,0.000000,100.000000
75%,228749.000000,1.000000,0.000000,100.000000
max,999595.000000,52.000000,0.000000,100.000000


In [21]:
identity_df['id_02'] = identity_df['id_02'].fillna(identity_df['id_02'].median())
identity_df['id_05'] = identity_df['id_05'].fillna(identity_df['id_05'].median())
identity_df['id_06'] = identity_df['id_06'].fillna(identity_df['id_06'].median())
identity_df['id_11'] = identity_df['id_11'].fillna(identity_df['id_11'].median())

#### 'id_02','id_05', 'id_06', 'id_11' columns nulls are filled with median()

In [38]:
nulls = identity_df.isnull().sum().sort_values(ascending =False)[identity_df.isnull().sum() > 0]
nulls

id_24         139486
id_25         139101
id_21         139074
id_26         139070
id_27         139064
id_22         139064
id_23         139064
id_18          99120
id_33          70944
id_30          66668
id_32          66647
id_34          66428
id_14          64189
DeviceInfo     25567
id_13          16913
id_16          14893
id_20           4972
id_19           4915
id_17           4864
id_31           3951
DeviceType      3423
id_28           3255
id_29           3255
id_15           3248
id_38           3248
id_37           3248
id_36           3248
id_35           3248
dtype: int64

In [39]:
nulls.index

Index(['id_24', 'id_25', 'id_21', 'id_26', 'id_27', 'id_22', 'id_23', 'id_18',
       'id_33', 'id_30', 'id_32', 'id_34', 'id_14', 'DeviceInfo', 'id_13',
       'id_16', 'id_20', 'id_19', 'id_17', 'id_31', 'DeviceType', 'id_28',
       'id_29', 'id_15', 'id_38', 'id_37', 'id_36', 'id_35'],
      dtype='object')

In [41]:
# fill "NI" No Information for categorical variables Nan vlues
# This creates another category for the variables
identity_df[nulls.index] = identity_df[nulls.index].fillna('NI')

'id_24', 'id_25', 'id_21', 'id_26', 'id_27', 'id_22', 'id_23', 'id_18',
       'id_33', 'id_30', 'id_32', 'id_34', 'id_14', 'DeviceInfo', 'id_13',
       'id_16', 'id_20', 'id_19', 'id_17', 'id_31', 'DeviceType', 'id_28',
       'id_29', 'id_15', 'id_38', 'id_37', 'id_36', 'id_35' columns are filled with 'NI' for Nan Values

### Check

In [43]:
# There is no null values in identity data frame anymore
identity_df.isnull().sum().sum()

0

## Save the indentity dataframe

In [45]:
identity_df.to_csv('C:\\Users\\chi\\Desktop\\spring_board\\capstone_2\\data\\identity_clean.csv', index=False)